<a href="https://colab.research.google.com/github/anonymous666666666/Assignment2-G24AI2059-AyushChauhan/blob/main/Assignment2_G24AI2059_AyushChauhan_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install mrjob and Java for Hadoop compatibility in Colab
!pip install mrjob
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download datasets using wget
!wget -O "cruise.csv" https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
!wget -O "customer_churn.csv" https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
!wget -O "ecom_customer.csv" https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 5.1 MB/s eta 0:00:00
--2025-07-26 14:50:50--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘cruise.csv’

cruise.csv          100%[===================>]   8.53K  --.-KB/s    in 0.001s  

2025-07-26 14:50:50 (15.4 MB/s) - ‘cruise.csv’ saved [8734/8734]

--2025-07-26 14:50:50--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [ ]:
#1. Cruiseline Aggregations (5 marks)
#Using cruise.csv, implement an mrjob class that computes, for each Cruise line:
#(a) Total number of ships.
#(b) Average Tonnage (to two decimals).
#(c) Maximum crew size.
#Optional) Use a Combiner for partial aggregation.


%%file CruiselineAggregation.py
from mrjob.job import MRJob
import csv
from io import StringIO

class CruiselineAggregation(MRJob):

    def mapper(self, _, line):
        reader = csv.reader(StringIO(line))
        row = next(reader)
        if row[0] == "Ship_name":
            return

        cruise_line = row[1]
        try:
            tonnage = float(row[3])
            crew = float(row[5])
        except ValueError:
            return

        yield cruise_line, (1, tonnage, crew)

    def combiner(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for count, tonnage, crew in values:
            total_ships += count
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)
        yield cruise_line, (total_ships, total_tonnage, max_crew)

    def reducer(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for count, tonnage, crew in values:
            total_ships += count
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)

        avg_tonnage = round(total_tonnage / total_ships, 2)
        yield cruise_line, {
            "Total Ships": total_ships,
            "Avg Tonnage": avg_tonnage,
            "Max Crew": max_crew
        }

if __name__ == '__main__':
    CruiselineAggregation.run()


Writing CruiselineAggregation.py


In [ ]:
!python CruiselineAggregation.py cruise.csv > q4_output.txt
!cat q4_output.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/CruiselineAggregation.root.20250726.145728.428034
Running step 1 of 1...
job output is in /tmp/CruiselineAggregation.root.20250726.145728.428034/output
Streaming final output from /tmp/CruiselineAggregation.root.20250726.145728.428034/output...
Removing temp directory /tmp/CruiselineAggregation.root.20250726.145728.428034...
"Cunard"	{"Total Ships": 3, "Avg Tonnage": 103.91, "Max Crew": 11.32}
"Disney"	{"Total Ships": 2, "Avg Tonnage": 83.17, "Max Crew": 9.64}
"Holland_American"	{"Total Ships": 14, "Avg Tonnage": 60.5, "Max Crew": 9.59}
"MSC"	{"Total Ships": 8, "Avg Tonnage": 63.77, "Max Crew": 10.93}
"Norwegian"	{"Total Ships": 13, "Avg Tonnage": 63.72, "Max Crew": 9.65}
"Oceania"	{"Total Ships": 3, "Avg Tonnage": 30.28, "Max Crew": 5.94}
"Orient"	{"Total Ships": 1, "Avg Tonnage": 22.08, "Max Crew": 5.78}
"P&O"	{"Total Ships": 6, "Avg Tonnage": 77.86, "Max Crew": 9.

In [ ]:
#2. Company Churn Rate (5 marks)
#From customer churn.csv, create a MultiStepJob:
#Step 1: Mapper emits (Company, TOTAL) and (Company, CHURNED) where Churn==1.
#Step 2: Reducer computes churn rate = CHURNED
#TOTAL , outputting four-decimal floats.
#Use a small VIP companies.txt in the distributed cache to restrict to listed companies.
#Provide a sample file with at least three names.


%%file vip_companies.txt
harvey llc
wilson plc
smith inc


Writing vip_companies.txt


In [ ]:
%%file CompanyChurnRate.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from io import StringIO

class CompanyChurnRate(MRJob):

    def configure_args(self):
        super().configure_args()
        self.add_file_arg('--vip_companies')

    def load_vip_companies(self):
        # Load and normalize VIP companies (lowercase and strip)
        with open(self.options.vip_companies, 'r') as f:
            return set(line.strip().lower() for line in f)

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   reducer=self.reducer_compute_churn_rate)
        ]

    def mapper_init(self):
        self.vip_set = self.load_vip_companies()

    def mapper(self, _, line):
        reader = csv.reader(StringIO(line))
        row = next(reader)

        # Skip header row or rows with invalid churn value
        try:
            churn = int(row[-1])
        except ValueError:
            return

        company = row[8].strip().lower()  # Normalize company name from CSV

        if company in self.vip_set:
            yield company, ('TOTAL', 1)
            if churn == 1:
                yield company, ('CHURNED', 1)

    def reducer_compute_churn_rate(self, company, values):
        total, churned = 0, 0
        for label, count in values:
            if label == 'TOTAL':
                total += count
            elif label == 'CHURNED':
                churned += count

        if total > 0:
            churn_rate = round(churned / total, 4)
            yield company, churn_rate

if __name__ == '__main__':
    CompanyChurnRate.run()


Writing CompanyChurnRate.py


In [ ]:
!python CompanyChurnRate.py "customer_churn.csv" --vip_companies vip_companies.txt > churn_output.txt

with open("churn_output.txt", "r") as f:
    print(f.read())


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/CompanyChurnRate.root.20250726.145930.290007
Running step 1 of 1...
job output is in /tmp/CompanyChurnRate.root.20250726.145930.290007/output
Streaming final output from /tmp/CompanyChurnRate.root.20250726.145930.290007/output...
Removing temp directory /tmp/CompanyChurnRate.root.20250726.145930.290007...
"wilson plc"	0.3333
"harvey llc"	1.0
"smith inc"	1.0



In [ ]:
#3. State-wise Spending (5 marks)
#From e-com customer.csv, extract the two-letter state code from the Address field.
#Then:
#• Mapper parses the state.
#• Reducer sums Yearly Amount Spent per state.
#• Output the top 5 states by total spending.



%%file StatewiseSpending.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from io import StringIO
from heapq import nlargest

class StatewiseSpending(MRJob):

    def mapper(self, _, line):
        reader = csv.reader(StringIO(line))
        row = next(reader)

        # Skip header
        if row[0] == "Email":
            return

        try:
            address = row[1].strip()  # Correct index for Address
            amount_spent = float(row[6])  # Correct index for Yearly Amount Spent
        except:
            return

        try:
            # Get state from the last comma-separated segment
            parts = address.split(',')
            if len(parts) < 2:
                return
            state_zip = parts[-1].strip().split()
            state = state_zip[0] if len(state_zip) > 0 else ""
        except IndexError:
            return

        if len(state) == 2:
            yield state, amount_spent

    def reducer(self, state, amounts):
        yield None, (sum(amounts), state)

    def reducer_find_top_5(self, _, state_amount_pairs):
        top5 = nlargest(5, state_amount_pairs)
        for amount, state in top5:
            yield state, round(amount, 2)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_find_top_5)
        ]

if __name__ == '__main__':
    StatewiseSpending.run()


Overwriting StatewiseSpending.py


In [ ]:
!python StatewiseSpending.py "ecom_customer.csv" > state_spending_output.txt

with open("state_spending_output.txt", "r") as f:
    print(f.read())


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/StatewiseSpending.root.20250726.150137.410588
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/StatewiseSpending.root.20250726.150137.410588/output
Streaming final output from /tmp/StatewiseSpending.root.20250726.150137.410588/output...
Removing temp directory /tmp/StatewiseSpending.root.20250726.150137.410588...
"SC"	53.64
"DE"	49.69
"MO"	45.63
"MN"	44.99
"VT"	44.22



In [ ]:
#4. Two-step Ship Filter & Median Length (5 marks)
#On cruise.csv, implement a two-step mrjob pipeline:
#Step 1: Filter ships with passenger density > 35.0; emit ⟨Cruise line, length⟩.
#Step 2: Compute the median of the lengths per Cruise line, handling even/odd counts
#correctly.
#Use the steps() API and output medians to two decimals.



%%file CruiseMedianLength.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from io import StringIO

class CruiseMedianLength(MRJob):

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper_filter_density,
                   reducer=self.reducer_collect_lengths),
            MRStep(reducer=self.reducer_compute_median)
        ]

    def mapper_init(self):
        self.header_processed = False

    def mapper_filter_density(self, _, line):
        reader = csv.reader(StringIO(line))
        row = next(reader)

        # Skip header row
        if not self.header_processed and "Ship_name" in row:
            self.header_processed = True
            return

        try:
            cruise_line = row[1]
            density = float(row[7])  # passenger_density
            length = float(row[5])   # length
        except:
            return

        if density > 35.0:
            yield cruise_line, length

    def reducer_collect_lengths(self, cruise_line, lengths):
        yield cruise_line, sorted(lengths)

    def reducer_compute_median(self, cruise_line, lengths_lists):
        all_lengths = []
        for lst in lengths_lists:
            all_lengths.extend(lst)

        all_lengths.sort()
        n = len(all_lengths)
        if n == 0:
            return

        if n % 2 == 1:
            median = all_lengths[n // 2]
        else:
            median = (all_lengths[n // 2 - 1] + all_lengths[n // 2]) / 2

        yield cruise_line, round(median, 2)

if __name__ == '__main__':
    CruiseMedianLength.run()


Overwriting CruiseMedianLength.py


In [ ]:
!python CruiseMedianLength.py cruise.csv > cruise_median_output.txt
with open("cruise_median_output.txt") as f:
    print(f.read())



No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/CruiseMedianLength.root.20250726.150234.249721
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/CruiseMedianLength.root.20250726.150234.249721/output
Streaming final output from /tmp/CruiseMedianLength.root.20250726.150234.249721/output...
Removing temp directory /tmp/CruiseMedianLength.root.20250726.150234.249721...
"Cunard"	9.64
"Disney"	9.64
"Holland_American"	7.77
"MSC"	8.23
"Norwegian"	9.0
"Oceania"	5.94
"P&O"	8.56
"Princess"	9.51
"Regent_Seven_Seas"	6.15
"Royal_Caribbean"	10.2
"Azamara"	5.94
"Carnival"	9.52
"Celebrity"	9.65
"Costa"	8.28
"Crystal"	7.86
"Seabourn"	4.4
"Silversea"	5.55
"Star"	2.8
"Windstar"	6.17

